# Plotters Tutorial with Jupyter

This is a interactive tutorial for [`Plotters`](https://github.com/38/plotters) drawing library. If you are looking at the static HTML version and want to try the interactive version. Please follow the steps:

#### For Ubuntu/Debian users

```bash
# Install Jupyter notebook 
sudo apt install libzmq3-dev jupyter-notebook
cargo install evcxr_jupyter
evcxr_jupyter --install
# Get the notebook
git clone https://github.com/38/plotters-doc-data
cd plotteres-doc-data
jupyter notebook
```

#### For OSX users

```bash
# Install Jupyter notebook 
brew install zeromq pkg-config
cargo install evcxr_jupyter
evcxr_jupyter --install
# Get the notebook
git clone https://github.com/38/plotters-doc-data
cd plotteres-doc-data
jupyter notebook
```

You can also download the latest notebook from [https://raw.githubusercontent.com/38/plotters-doc-data/master/evcxr-jupyter-integration.ipynb](https://raw.githubusercontent.com/38/plotters-doc-data/master/evcxr-jupyter-integration.ipynb), thus you don't have to clone the entire data repo.

## Get Started

In order to use `Plotters` in `jupyter-evcxr`, you need both Jupyter and evcxr installed.
Check [https://github.com/google/evcxr](https://github.com/google/evcxr) for the instructions.

To use Plotters with `jupyter-evcxr`, you need to import it using the following code:

In [2]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }

Because `evcxr` uses only SVG images and all types of series, so we don't need other types of backend. So we should put

`default_features = false, features = ["evcxr", "all_series"]`

Make the compilation faster. Since `evcxr` shares all the artifacts among cells, after the first time we have `plotters` compiled, it should be faster after.

### Plotters evcxr integration overview

To use plotters, the most convenient way is importing everything defined in the `prelude` module.
It will import `evcxr_figure` function for `evcxr` integration. 

*Note: Currently evcxr doesn't work with nightly rust, so please make sure you are using a stable rust*

In [4]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }
extern crate plotters;
// Import all the plotters prelude functions
use plotters::prelude::*;
// To create a figure that can be displayed in Jupyter notebook, use evcxr_figure function.
// The first param is the resolution of the figure.
// The second param is the closure that performes the drawing.
evcxr_figure((300, 100), |root| {
    // Do the drawings
    root.fill(&BLUE)?;
    // Tell plotters that everything is ok
    Ok(())
})

## Hello World

In [5]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }
extern crate plotters;
use plotters::prelude::*;

evcxr_figure((320,50), |root| {
    root.fill(&GREEN)?;
    root.draw(&Text::new("Hello World from Plotters!", (15, 15), ("Arial", 20).into_font()))?;
    Ok(())
})

Hello World from Plotters!

## Sub- Drawing Areas

One of the very important features is, `Plotters` allows drawing multiple charts in a single figure. And this is done by having sub-drawing-areas. The root drawing area is able to be splitted into smaller drawing areas, and you can always do more fine-grained splits as well.

In [6]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }
extern crate plotters;
use plotters::prelude::*;
use plotters::coord::Shift;
pub fn sierpinski_carpet(
    depth: u32, 
    drawing_area: &DrawingArea<SVGBackend, Shift>) 
-> Result<(), Box<dyn std::error::Error>> {
    if depth > 0 {
        let sub_areas = drawing_area.split_evenly((3,3));
        for (idx, sub_area) in (0..).zip(sub_areas.iter()) {
            if idx == 4 {
                sub_area.fill(&WHITE)?;
            } else {
                sierpinski_carpet(depth - 1, sub_area)?;
            }
        }
    }
    Ok(())
}
evcxr_figure((4800,4800), |root| {
    root.fill(&BLACK)?;
    sierpinski_carpet(5, &root)
}).style("width: 200px")  /* You can add CSS style to the result */


<rect x="614" y="3043

## Chart Context

`Plotters` is designed for drawing charts, plots, etc. This example demonstrate how to use `Plotters` chart specific APIs to draw a chart, including, labels, axis, meshes, etc. To draw a chart on the drawin area, you need to create a chart context and do some configuration.

In [7]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }
extern crate plotters;
use plotters::prelude::*;
evcxr_figure((640, 240), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Hello Plotters Chart Context!", ("Arial", 20).into_font())
        .build_cartesian_2d(0f32..1f32, 0f32..1f32)?;
    // Then we can draw a series on it!
    chart.draw_series((1..10).map(|x|{
        let x = x as f32/10.0;
        Circle::new((x,x), 5, &RED)
    }))?;
    Ok(())
}).style("width:60%")

Hello Plotters Chart Context!

## Adding Common Chart Components 

We can also make `Plotters` draws common components for us, such as, meshes, axis, legend. In this section, we demonstrate how to do that.

The following code shows how we add mesh to the chart.

In [8]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }
extern crate plotters;
use plotters::prelude::*;
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Chart Context with Mesh", ("Arial", 20).into_font())
        .build_cartesian_2d(0f32..1f32, 0f32..1f32)?;
    chart.configure_mesh().draw()?;
    Ok(())
}).style("width: 60%")

Chart Context with Mesh

Then we can add axis to the chart.

In [9]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }
extern crate plotters;
use plotters::prelude::*;
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Chart Context with Mesh and Axis", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f32..1f32, 0f32..1f32)?;
    
    chart.configure_mesh()
        .draw()?;
    
    Ok(())
}).style("width: 60%")

Chart Context with Mesh and Axis
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0

In addition to that, we can put label text to the axis.

In [10]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }
extern crate plotters;
use plotters::prelude::*;
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Chart with Axis Label", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f32..1f32, 0f32..1f32)?;
    
    chart.configure_mesh()
        .x_desc("Here's the label for X")
        .y_desc("Here's the label for Y")
        .draw()?;
    
    Ok(())
}).style("width: 60%")

Chart with Axis Label
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Here's the label for Y
 
 
Here's the label for X
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0

Then let's disable mesh lines for the X axis

In [12]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }
extern crate plotters;
use plotters::prelude::*;
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Chart Context with Mesh and Axis", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f32..1f32, 0f32..1f32)?;
    
    chart.configure_mesh()
        .y_labels(10)
        .light_line_style(&TRANSPARENT)
        .disable_x_mesh()
        .draw()?;
    
    Ok(())
}).style("width: 60%")

Chart Context with Mesh and Axis
 
 
 
 
 
 
 
 
 
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0

To create multiple charts in a single figure, you can just split the drawing area and create multiple chart context.

In [13]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }
extern crate plotters;
use plotters::prelude::*;
evcxr_figure((640, 480), |root| {
    let sub_areas = root.split_evenly((2,2));
    
    for (idx, area) in (1..).zip(sub_areas.iter()) {
        // The following code will create a chart context
        let mut chart = ChartBuilder::on(&area)
            .caption(format!("Subchart #{}", idx), ("Arial", 15).into_font())
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_cartesian_2d(0f32..1f32, 0f32..1f32)?;

        chart.configure_mesh()
            .y_labels(10)
            .light_line_style(&TRANSPARENT)
            .disable_x_mesh()
            .draw()?;
    }

    Ok(())
}).style("width: 60%")

Subchart #1
 
 
 
 
 
 
 
 
 
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
Subchart #2
 
 
 
 
 
 
 
 
 
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
Subchart #3
 
 
 
 
 
 
 
 
 
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
Subchart #4
 
 
 
 
 
 
 
 
 
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0

## Series

Unlike most of the plotting libraries, `Plotters` doesn't actually define any types of chart. All the chart is abstracted to a concept of series. By doing so, you can put a histgoram series and a line plot series into the same chart context.
The series is actually defined as an iterator of elements, just this.

This gives `Plotters` a huge flexibility on drawing charts. You can implement you own types of series and uses the coordinate translation and chart elements. 

There are few types of predefined series, just for convenience:
- Line Series
- Histogram
- Point Series

### Scatter Plot

First of all, let's generate some random numbers.

In [14]:
:dep rand = { version = "0.6.5" }
extern crate rand;

use rand::distributions::Normal;
use rand::distributions::Distribution;
use rand::thread_rng;
let sd = 0.13;
let random_points:Vec<(f64,f64)> = {
    let mut norm_dist = Normal::new(0.5, sd);
    let (mut x_rand, mut y_rand) = (thread_rng(), thread_rng());
    let x_iter = norm_dist.sample_iter(&mut x_rand);
    let y_iter = norm_dist.sample_iter(&mut y_rand);
    x_iter.zip(y_iter).take(1000).collect()
};
random_points.len()

1000

It's trivial to draw a scatter plot with `Plotters`. The only need is, provide a iterator of the elements as series.
The following example shows how to make a 2D normal distribution figure. The red rectangle is the two sigma area and the red cross is the mean.

In [15]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }

extern crate plotters;
use plotters::prelude::*;

evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Normal Distribution", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_ranged(0f64..1f64, 0f64..1f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    
    chart.draw_series(random_points.iter().map(|(x,y)| Circle::new((*x,*y), 3, GREEN.filled())));
    
    // You can alawys freely draw on the drawing backend
    let area = chart.plotting_area();
    let two_sigma = sd * 2.0;
    area.draw(&Rectangle::new(
        [(0.5 - two_sigma, 0.5 - two_sigma), (0.5 + two_sigma, 0.5 + two_sigma)], 
        RED.mix(0.3).filled())
    )?;
    area.draw(&Cross::new((0.5, 0.5), 5, &RED))?;
    
    Ok(())
}).style("width:60%")

Normal Distribution
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0

### Histogram

We can also have histograms. For histograms, we can use the predefined histogram series struct to build the histogram easily. The following code demonstrate how to create both histogram for X and Y value of `random_points`.

In [18]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "all_elements"] }

extern crate plotters;
use plotters::prelude::*;

evcxr_figure((640, 480), |root| {
    let areas = root.split_evenly((2,1));
    let mut charts = vec![];
    
    // The following code will create a chart context
   for (area, name) in areas.iter().zip(["X", "Y"].into_iter()) {
        let mut chart = ChartBuilder::on(&area)
            .caption(format!("Histogram for {}", name), ("Arial", 20).into_font())
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_cartesian_2d(0u32..100u32, 0f64..0.5f64)?;
        chart.configure_mesh()
            .disable_x_mesh()
            .disable_y_mesh()
            .y_labels(5)
            .x_label_formatter(&|x| format!("{:.1}", *x as f64 / 100.0))
            .y_label_formatter(&|y| format!("{}%", (*y * 100.0) as u32))
            .draw()?;
        charts.push(chart);
    }
    
    let hist_x = Histogram::vertical(&charts[0])
        .style(RED.filled())
        .margin(0)
        .data(random_points.iter().map(|(x,_)| ((x*100.0) as u32, 0.01)));
    
    let hist_y = Histogram::vertical(&charts[0])
        .style(GREEN.filled())
        .margin(0)
        .data(random_points.iter().map(|(_,y)| ((y*100.0) as u32, 0.01)));
    
    charts[0].draw_series(hist_x);
    charts[1].draw_series(hist_y);
    
    Ok(())
}).style("width:60%")

Histogram for X
 
 
 
10%
 
 
 
20%
 
 
 
30%
 
 
 
40%
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
Histogram for Y
 
 
 
10%
 
 
 
20%
 
 
 
30%
 
 
 
40%
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0

### Combination of Histogram and Scatter

In [26]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "all_elements"] }

use plotters::prelude::*;

evcxr_figure((640, 480), |root| {
    let root = root.titled("Scatter with Histogram Example", ("Arial", 20).into_font())?;
    
    let areas = root.split_by_breakpoints([560], [80]);

    let mut x_hist_ctx = ChartBuilder::on(&areas[0])
        .y_label_area_size(40)
        .build_cartesian_2d(0u32..100u32, 0f64..0.5f64)?;
    let mut y_hist_ctx = ChartBuilder::on(&areas[3])
        .x_label_area_size(40)
        .build_cartesian_2d(0f64..0.5f64, 0..100u32)?;
    let mut scatter_ctx = ChartBuilder::on(&areas[2])
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..1f64, 0f64..1f64)?;
    scatter_ctx.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    scatter_ctx.draw_series(random_points.iter().map(|(x,y)| Circle::new((*x,*y), 3, GREEN.filled())))?;
    let x_hist = Histogram::vertical(&x_hist_ctx)
        .style(RED.filled())
        .margin(0)
        .data(random_points.iter().map(|(x,_)| ((x*100.0) as u32, 0.01)));
    let y_hist = Histogram::horizontal(&y_hist_ctx)
        .style(GREEN.filled())
        .margin(0)
        .data(random_points.iter().map(|(_,y)| ((y*100.0) as u32, 0.01)));
    x_hist_ctx.draw_series(x_hist)?;
    y_hist_ctx.draw_series(y_hist)?;
    
    Ok(())
}).style("width:60%")

Scatter with Histogram Example
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0

## 3D Plots

Plotters also support 3D plotting since 0.3. To plotting a 3D chart, it's quite simple: when constructing the `ChartContext`, instead of `build_cartesian_2d`, use `build_cartesian_3d` instead. Then you should get a plotting environment that supports accepts 3-dimensional coordinates.

In [41]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
use plotters::prelude::*;

evcxr_figure((640, 480), |root| {
    let root = root.titled("3D Plotting", ("Arial", 20).into_font())?;
    
    let mut chart = ChartBuilder::on(&root)
        .build_cartesian_3d(-10.0..10.0, -10.0..10.0, -10.0..10.0)?;
    // Draw a red circle parallel to XOZ panel
    chart.draw_series(LineSeries::new(
        (-314..314).map(|a| a as f64 / 100.0).map(|a| (8.0 * a.cos(), 0.0, 8.0 *a.sin())),
        &RED,
    ))?;
    // Draw a green circle parallel to YOZ panel
    chart.draw_series(LineSeries::new(
        (-314..314).map(|a| a as f64 / 100.0).map(|a| (0.0, 8.0 * a.cos(), 8.0 *a.sin())),
        &GREEN,
    ))?;
    
    Ok(())
})

3D Plotting
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="192,242 193,242 194,242 195,241 195,241 196,241 197,241 197,241 198,241 198,240 200,240 201,240 202,240 202,240 203,240 204,239 205,239 206,239 206,239 207,239 209,239 209,238 210,238 212,238 213,238 213,238 215,237 215,237 217,237 218,237 218,237 220,237 220,237 222,237 223,236 224,236 225,236 226,236 227,236 228,236 229,235 231,235 232,235 233,235 234,235 235,235 237,235 238,235 238,234 240,234 242,234 242,234 243,234 245,234 247,234 247,234 248,234 250,233 252,233 252,233 254,233 255,233 257,233 258,233 259,233 261,233 261,233 263,233 264,232 266,232 267,232 269,232 270,232 271,232 273,232 274,232 275,232 277,232 278,232 279,232 281,232 282,232 283,232 285,231 286,231 288,231 289,231 291,231 292,231 294,231 295,231 297,231 298,231 299,231 301,231 302,231 304,231 305,231 306,231 309,231 310,231 311,231 312,231 314,231 315,231 317,231 318,231 319,231 321,231 323,231 324,231 326,231 327,231 328,231 330,231 331,231 332,231 335,231 335,231 337,231 339,231 340,231 341,231 343,231 344,231 345,231 347,231 348,231 350,231 352,231 352,231 354,231 356,231 357,232 358,232 359,232 361,232 362,232 364,232 365,232 367,232 368,232 369,232 371,232 372,232 374,232 374,232 376,233 378,233 379,233 380,233 381,233 383,233 384,233 386,233 386,233 388,233 389,233 391,234 392,234 393,234 394,234 395,234 397,234 398,234 400,234 400,234 402,235 403,235 403,235 405,235 406,235 408,235 408,235 410,236 411,236 413,236 413,236 414,236 416,236 416,236 418,237 419,237 419,237 421,237 422,237 423,237 424,237 424,238 426,238 426,238 428,238 429,238 429,238 431,239 432,239 432,239 434,239 434,239 435,239 436,240 437,240 438,240 439,240 439,240 440,240 441,241 442,241 443,241 443,241 444,241 445,242 446,242 446,242 446,242 448,242 449,242 449,243 449,243 450,243 451,243 451,244 452,244 453,244 453,244 454,244 455,244 455,245 456,245 456,245 456,245 457,245 457,246 458,246 458,246 458,246 459,247 459,247 460,247 460,247 460,247 461,248 461,248 462,248 462,248 462,248 462,249 463,249 463,249 463,249 463,249 463,250 463,250 464,250 464,250 465,251 464,251 464,251 464,251 465,251 465,252 464,252 465,252 465,252 465,252 464,253 465,253 464,253 464,253 465,254 464,254 465,254 465,254 464,254 464,255 463,255 464,255 463,255 463,256 463,256 463,256 462,256 463,256 462,257 462,257 462,257 462,257 461,257 460,258 460,258 460,258 459,258 459,258 458,259 458,259 458,259 457,259 457,259 456,260 456,260 455,260 455,260 454,261 454,261 453,261 453,261 453,261 452,262 451,262 450,262 450,262 449,262 448,262 447,263 447,263 446,263 445,263 445,263 444,264 443,264 442,264 442,264 441,264 441,265 439,265 438,265 438,265 437,265 436,265 435,266 434,266 433,266 433,266 432,266 430,266 430,267 429,267 427,267 426,267 426,267 425,267 424,268 423,268 421,268 421,268 420,268 419,268 417,268 417,269 415,269 414,269 413,269 412,269 411,269 410,270 408,270 408,270 406,270 405,270 404,270 403,270 401,270 401,271 399,271 397,271 397,271 396,271 394,271 393,271 392,271 391,271 389,271 387,272 387,272 385,272 384,272 383,272 381,272 380,272 379,272 378,272 376,272 375,273 374,273 372,273 370,273 369,273 368,273 366,273 365,273 364,273 362,273 361,273 360,273 358,273 357,273 356,273 354,274 353,274 351,274 350,274 349,274 347,274 345,274 344,274 342,274 341,274 340,274 339,274 337,274 335,274 334,274 333,274 332,274 329,274 328,274 327,274 326,274 325,274 322,274 321,274 320,274 318,274 316,274 316,274 314,274 312,274 311,274 310,274 308,274 307,274 305,274 304,274 302,274 301,274 299,274 298,274 297,274 295,274 294,274 292,274 291,274 290,274 287,274 287,274 285,274 283,274 283,273 281,273 280,273 278,273 277,273 276,273 274,273 273,273 271,273 270,273 268,273 267,273 265,273 265,273 264,272 261,272 261,272 259,272 258,272 256,272 255,272 253,272 253,272 252,272 250,272 249,271 247,271 246,271 245,271 244,271 242,271 241,271 240,271 239,271 238,270 236,270 236,270 234,270 233,270 231,270 231,270 230,27

### Draw Plot Components 

Just like the 2D plots, we are able to draw the axis and axis panel easily. Unlike the 2D plots, 3D plots use the function `configure_axes` to configure the chart components.

In [42]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
use plotters::prelude::*;

evcxr_figure((640, 480), |root| {
    let root = root.titled("3D Plotting", ("Arial", 20).into_font())?;
    
    let mut chart = ChartBuilder::on(&root)
        .build_cartesian_3d(-10.0..10.0, -10.0..10.0, -10.0..10.0)?;
    
    chart.configure_axes().draw()?;
    
    // Draw a red circle parallel to XOZ panel
    chart.draw_series(LineSeries::new(
        (-314..314).map(|a| a as f64 / 100.0).map(|a| (8.0 * a.cos(), 0.0, 8.0 *a.sin())),
        &RED,
    ))?;
    // Draw a green circle parallel to YOZ panel
    chart.draw_series(LineSeries::new(
        (-314..314).map(|a| a as f64 / 100.0).map(|a| (0.0, 8.0 * a.cos(), 8.0 *a.sin())),
        &GREEN,
    ))?;
    
    Ok(())
})

3D Plotting
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-8.0
 
 
 
-6.0
 
 
 
-4.0
 
 
 
-2.0
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
 
-8.0
 
 
 
-6.0
 
 
 
-4.0
 
 
 
-2.0
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
 
-8.0
 
 
 
-6.0
 
 
 
-4.0
 
 
 
-2.0
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="192,242 193,242 194,242 195,241 195,241 196,241 197,241 197,241 198,241 198,240 200,240 201,240 202,240 202,240 203,240 204,239 205,239 206,239 206,239 207,239 209,239 209,238 210,238 212,238 213,238 213,238 215,237 215,237 217,237 218,237 218,237 220,237 220,237 222,237 223,236 224,236 225,236 226,236 227,236 228,236 229,235 231,235 232,235 233,235 234,235 235,235 237,235 238,235 238,234 240,234 242,234 242,234 243,234 245,234 247,234 247,234 248,234 250,233 252,233 252,233 254,233 255,233 257,233 258,233 259,233 261,233 261,233 263,233 264,232 266,232 267,232 269,232 270,232 271,232 273,232 274,232 275,232 277,232 278,232 279,232 281,232 282,232 283,232 285,231 286,231 288,231 289,231 291,231 292,231 294,231 295,231 297,231 298,231 299,231 301,231 302,231 304,231 305,231 306,231 309,231 310,231 311,231 312,231 314,231 315,231 317,231 318,231 319,231 321,231 323,231 324,231 326,231 327,231 328,231 330,231 331,231 332,231 335,231 335,231 337,231 339,231 340,231 341,231 343,231 344,231 345,231 347,231 348,231 350,231 352,231 352,231 354,231 356,231 357,232 358,232 359,232 361,232 362,232 364,232 365,232 367,232 368,232 369,232 371,232 372,232 374,232 374,232 376,233 378,233 379,233 380,233 381,233 383,233 384,233 386,233 386,233 388,233 389,233 391,234 392,234 393,234 394,234 395,234 397,234 398,234 400,234 400,234 402,235 403,235 403,235 405,235 406,235 408,235 408,235 410,236 411,236 413,236 413,236 414,236 416,236 416,236 418,237 419,237 419,237 421,237 422,237 423,237 424,237 424,238 426,238 426,238 428,238 429,238 429,238 431,239 432,239 432,239 434,239 434,239 435,239 436,240 437,240 438,240 439,240 439,240 440,240 441,241 442,241 443,241 443,241 444,241 445,242 446,242 446,242 446,242 448,242 449,242 449,243 449,243 450,243 451,243 451,244 452,244 453,244 453,244 454,244 455,244 455,245 456,245 456,245 456,245 457,245 457,246 458,246 458,246 458,246 459,247 459,247 460,247 460,247 460,247 461,248 461,248 462,248 462,248 462,248 462,249 463,249 463,249 463,249 463,249 463,250 463,250 464,250 464,250 465,251 464,251 464,251 464,251 465,251 465,252 464,252 465,252 465,252 465,252 464,253 465,253 464,253 464,253 465,254 464,254 465,254 465,254 464,254 464,255 463,255 464,255 463,255 463,256 463,256 463,256 462,256 463,256 462,257 462,257 462,257 462,257 461,257 460,258 460,258 460,258 459,258 459,258 458,259 458,259 458,259 457,259 457,259 456,260 456,260 455,260 455,260 454,261 454,261 453,261 453,261 453,261 452,262 451,262 450,262 450,262 449,262 448,262 447,263 447,263 446,263 445,263 445,263 444,264 443,264 442,264 442,264 441,264 441,265 439,265 438,265 438,265 437,265 436,265 435,266 434,266 433,266 433,266 432,266 430,266 430,267 429,267 427,267 426,267 426,267 425,267 424,268 423,268 421,268 421,268 420,268 419,268 417,268 417,269 415,269 414,269 413,269 412,269 411,269 410,270 408,270 408,270 406,270 405,270 404,270 403,270 401,270 401,271 399,271 397,271 397,271 396,271 394,271 393,271 392,271 391,271 389,271 387,272 387,272 385,272 384,272 383,272 381,272 380,272 379,272 378,272 376,272 375,273 374,273 372,273 370,273 369,273 368,273 366,273 365,273 364,273 362,273 361,273 360,273 358,273 357,273 356,273 354,274 353,274 351,274 350,274 349,274 347,274 345,274 344,274 342,274 341,274 340,274 339,274 337,274 335,274 334,274 333,274 332,274 329,274 328,274 327,274 326,274 325,274 322,274 321,274 320,274 318,274 316,274 316,274 314,274 312,274 311,274 310,274 308,274 307,274 305,274 3

### Adjusting the 3D View

Unlike the 2D plot, 3D plot allows you to change the point of view. For example, rotate the plot or scale the plot, etc. This is done by the projection matrix, Plotters allows you override the default projection matrix, so that you can customize your view point of the 3D Plot. The following example shows how to look at the plot from top to bottom in the plot we have previously created.

In [47]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
use plotters::prelude::*;

evcxr_figure((640, 480), |root| {
    let root = root.titled("3D Plotting", ("Arial", 20).into_font())?;
    
    let mut chart = ChartBuilder::on(&root)
        .build_cartesian_3d(-10.0..10.0, -10.0..10.0, -10.0..10.0)?;
    chart.with_projection(|mut p| {
        p.pitch = 1.5707; // 90 degreen pitch, thus we are looking the plot from top
        p.yaw = 0.0;      // Make plot's X axis parallel to screen's X axis    
        p.into_matrix() // build the projection matrix
    });
    
    chart.configure_axes().draw()?;
    
    // Draw a red circle parallel to XOZ panel
    chart.draw_series(LineSeries::new(
        (-314..314).map(|a| a as f64 / 100.0).map(|a| (8.0 * a.cos(), 0.0, 8.0 *a.sin())),
        &RED,
    ))?;
    // Draw a green circle parallel to YOZ panel
    chart.draw_series(LineSeries::new(
        (-314..314).map(|a| a as f64 / 100.0).map(|a| (0.0, 8.0 * a.cos(), 8.0 *a.sin())),
        &GREEN,
    ))?;
    
    Ok(())
})

3D Plotting
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-8.0
 
 
 
-6.0
 
 
 
-4.0
 
 
 
-2.0
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
 
-8.0
 
 
 
-6.0
 
 
 
-4.0
 
 
 
-2.0
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
 
-8.0
 
 
 
-6.0
 
 
 
-4.0
 
 
 
-2.0
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="175,253 175,251 175,250 175,248 175,247 175,246 175,244 175,243 175,241 175,240 176,238 176,237 176,235 176,234 176,233 176,231 177,230 177,228 177,227 177,225 178,224 178,223 178,221 179,220 179,218 179,217 180,215 180,214 181,213 181,211 181,210 182,208 182,207 183,206 183,204 184,203 184,202 185,200 185,199 186,198 186,196 187,195 188,194 188,192 189,191 189,190 190,188 191,187 191,186 192,184 193,183 193,182 194,181 195,179 196,178 196,177 197,176 198,174 199,173 199,172 200,171 201,170 202,168 203,167 204,166 205,165 205,164 206,163 207,162 208,160 209,159 210,158 211,157 212,156 213,155 214,154 215,153 216,152 217,151 218,150 219,149 220,148 221,147 222,146 223,145 224,144 225,143 227,142 228,141 229,140 230,139 231,138 232,137 233,136 235,136 236,135 237,134 238,133 239,132 241,131 242,131 243,130 244,129 245,128 247,128 248,127 249,126 250,126 252,125 253,124 254,123 256,123 257,122 258,122 260,121 261,120 262,120 264,119 265,119 266,118 268,118 269,117 270,117 272,116 273,116 274,115 276,115 277,114 279,114 280,113 281,113 283,113 284,112 286,112 287,112 288,111 290,111 291,111 293,110 294,110 296,110 297,110 298,109 300,109 301,109 303,109 304,109 306,109 307,108 309,108 310,108 311,108 313,108 314,108 316,108 317,108 319,108 320,108 322,108 323,108 324,108 326,108 327,108 329,108 330,108 332,108 333,108 335,109 336,109 337,109 339,109 340,109 342,109 343,110 345,110 346,110 348,110 349,111 350,111 352,111 353,112 355,112 356,112 357,113 359,113 360,113 362,114 363,114 364,115 366,115 367,116 369,116 370,117 371,117 373,118 374,118 375,119 377,119 378,120 379,120 381,121 382,122 383,122 385,123 386,124 387,124 388,125 390,126 391,126 392,127 394,128 395,129 396,129 397,130 398,131 400,132 401,132 402,133 403,134 404,135 406,136 407,137 408,137 409,138 410,139 411,140 413,141 414,142 415,143 416,144 417,145 418,146 419,147 420,148 421,149 422,150 423,151 424,152 425,153 426,154 427,155 428,156 429,157 430,158 431,159 432,161 433,162 434,163 435,164 436,165 436,166 437,167 438,169 439,170 440,171 441,172 441,173 442,175 443,176 444,177 445,178 445,180 446,181 447,182 447,183 448,185 449,186 449,187 450,189 451,190 451,191 452,192 453,194 453,195 454,196 454,198 455,199 455,200 456,202 456,203 457,205 457,206 458,207 458,209 459,210 459,211 460,213 460,214 460,216 461,217 461,218 461,220 462,221 462,223 462,224 463,226 463,227 463,228 463,230 464,231 464,233 464,234 464,236 464,237 464,239 465,240 465,241 465,243 465,244 465,246 465,247 465,249 465,250 465,252 465,253 465,253 465,255 465,256 465,258 465,259 465,261 465,262 465,264 465,265 464,266 464,268 464,269 464,271 464,272 464,274 463,275 463,277 463,278 463,279 462,281 462,282 462,284 461,285 461,287 461,288 460,289 460,291 460,292 459,294 459,295 458,296 458,298 457,299 457,300 456,302 456,303 455,305 455,306 454,307 454,309 453,310 453,311 452,313 451,314 451,315 450,316 449,318 449,319 448,320 447,322 447,323 446,324 445,325 445,327 444,328 443,329 442,330 441,332 441,333 440,334 439,335 438,336 437,338 436,339 436,340 435,341 434,342 433,343 432,344 431,346 430,347 429,348 428,349 427,350 426,351 425,352 424,353 423,354 422,355 421,356 420,357 419,358 418,359 417,360 416,361 415,362 414,363 413,364 411,365 410,366 409,367 408,368 407,368 406,369 404,370 403,371 402,372 401,373 400,373 398,374 397,375 396,376 395,376 394,377 392,378 391,379 390,379 388,380 387,381 386,381 385,382 383,383 382,383 381,384 379,385 378,385 377,386 3

In [48]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
use plotters::prelude::*;

evcxr_figure((640, 480), |root| {
    let root = root.titled("3D Plotting", ("Arial", 20).into_font())?;
    
    let mut chart = ChartBuilder::on(&root)
        .build_cartesian_3d(-10.0..10.0, -10.0..10.0, -10.0..10.0)?;
    chart.with_projection(|mut p| {
        p.pitch = 0.7; 
        p.yaw = 0.7;
        p.scale = 0.5;
        p.into_matrix() // build the projection matrix
    });
    
    chart.configure_axes().draw()?;
    
    // Draw a red circle parallel to XOZ panel
    chart.draw_series(LineSeries::new(
        (-314..314).map(|a| a as f64 / 100.0).map(|a| (8.0 * a.cos(), 0.0, 8.0 *a.sin())),
        &RED,
    ))?;
    // Draw a green circle parallel to YOZ panel
    chart.draw_series(LineSeries::new(
        (-314..314).map(|a| a as f64 / 100.0).map(|a| (0.0, 8.0 * a.cos(), 8.0 *a.sin())),
        &GREEN,
    ))?;
    
    Ok(())
})

3D Plotting
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-8.0
 
 
 
-6.0
 
 
 
-4.0
 
 
 
-2.0
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
 
-8.0
 
 
 
-6.0
 
 
 
-4.0
 
 
 
-2.0
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
 
-8.0
 
 
 
-6.0
 
 
 
-4.0
 
 
 
-2.0
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="264,222 265,222 265,222 266,221 266,221 266,221 267,220 267,220 268,219 268,219 269,219 270,219 270,218 271,218 271,218 272,217 272,217 273,217 273,216 274,216 275,216 275,216 276,215 276,215 277,215 277,214 278,214 279,214 279,214 280,213 280,213 281,213 282,213 282,212 283,212 284,212 284,212 285,211 285,211 286,211 287,211 287,211 288,211 289,210 289,210 290,210 291,210 291,209 292,209 293,209 293,209 294,209 295,209 296,208 296,208 297,208 297,208 298,208 299,208 299,207 300,207 301,207 302,207 302,207 303,207 304,207 304,207 305,207 306,207 307,206 307,206 308,206 309,206 309,206 310,206 311,206 312,206 312,206 313,206 314,206 314,206 315,206 316,206 316,206 317,206 318,206 319,206 320,206 320,206 321,206 322,206 323,206 323,206 324,206 325,206 325,206 326,206 327,206 327,206 329,206 329,206 330,206 330,206 331,206 332,207 333,207 333,206 334,207 335,207 335,207 336,207 337,207 338,207 338,207 339,208 340,207 340,208 341,208 342,208 342,208 343,208 344,208 344,209 345,209 346,209 346,209 347,209 348,209 349,210 349,210 350,210 350,210 351,210 352,211 352,211 353,211 354,211 354,211 355,212 356,212 356,212 357,212 357,212 358,213 359,213 359,213 360,214 361,214 361,214 362,214 362,215 363,215 364,215 364,216 365,216 365,216 366,217 366,217 367,217 367,217 368,218 369,218 369,218 370,219 370,219 371,219 371,219 372,220 372,220 372,221 373,221 374,221 374,222 374,222 375,222 376,223 376,223 376,224 377,224 377,224 378,225 378,225 379,225 379,226 380,226 380,226 380,227 381,227 381,228 382,228 382,229 382,229 382,229 383,230 383,230 384,230 384,231 384,231 385,232 385,232 385,233 385,233 386,233 386,234 386,234 387,235 387,235 387,236 388,236 388,236 388,237 388,238 389,238 389,238 389,239 389,239 389,239 390,240 390,240 390,241 390,242 390,242 391,242 391,243 391,243 391,244 391,244 391,245 391,245 391,246 391,246 391,246 392,247 392,247 392,248 392,248 392,249 392,249 392,250 392,250 392,251 392,251 392,251 392,252 392,252 392,253 392,254 392,254 392,254 392,255 392,255 392,256 392,256 392,257 392,257 392,258 392,258 391,259 391,259 391,260 391,260 391,260 391,261 391,261 391,262 390,262 390,263 390,263 390,264 390,264 390,264 390,265 389,266 389,266 389,266 388,267 389,267 388,267 388,268 388,268 387,269 387,269 387,270 386,270 387,271 386,271 386,272 386,272 385,272 385,273 385,273 384,274 384,274 383,275 383,275 383,276 383,276 382,276 382,277 382,277 381,277 381,278 380,278 380,278 379,279 379,279 379,280 378,280 378,280 377,281 377,281 376,282 376,282 375,282 375,283 375,283 374,283 374,284 373,284 373,284 372,285 372,285 371,286 371,286 370,286 369,286 369,287 368,287 368,287 367,288 367,288 366,288 366,289 365,289 364,289 364,289 363,290 363,290 362,290 362,291 361,291 360,291 360,291 359,292 359,292 358,292 357,292 357,293 356,293 355,293 355,293 354,294 354,294 353,294 352,294 352,294 351,295 350,295 350,295 349,295 349,295 348,296 347,296 347,296 346,296 345,296 344,296 344,296 343,297 342,297 342,297 341,297 340,297 340,298 339,298 338,298 338,298 337,298 336,298 336,298 335,298 334,298 333,298 333,298 332,299 331,299 330,299 330,299 329,299 328,299 327,299 327,299 326,299 325,299 325,299 324,299 323,299 323,299 322,299 321,299 320,299 320,299 319,299 318,299 317,299 316,299 316,299 315,299 315,299 314,299 313,299 312,299 312,299 311,299 310,299 309,299 309,299 308,299 308,299 306,298 306,299 305,298 304,298 304,298 303,298 302,298 301,298 301,298 300,298 299,298 299,297 298,297 2

### A 2D Guassian Distribution Example

In [124]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
use plotters::prelude::*;
fn pdf(x: f64, y: f64) -> f64 {
    const SDX: f64 = 0.1;
    const SDY: f64 = 0.1;
    const A: f64 = 5.0;
    let x = x as f64 / 10.0;
    let y = y as f64 / 10.0;
    A * (- x * x / 2.0 / SDX / SDX - y * y / 2.0 / SDY / SDY).exp()
}
evcxr_figure((640 * 2, 480), |root| {

    let root = root.titled("2D Guassian PDF", ("Arial", 20).into_font())?;
    
    let (left, right) = root.split_horizontally(640);

    for (pitch, yaw, area) in vec![(0.6, 0.3, left), (1.5707, 0.0, right)] {
        let mut chart = ChartBuilder::on(&area)
            .build_cartesian_3d(-3.0..3.0, 0.0..6.0, -3.0..3.0)?;
        chart.with_projection(|mut p| {
            p.pitch = pitch; 
            p.yaw = yaw;
            p.scale = 0.7;
            p.into_matrix() // build the projection matrix
        });

        chart.configure_axes().draw()?;

        let series = (-15..15).map(|x| std::iter::repeat(x).zip(-15..15)).flatten().map(|(x, z)|{
            let x = x as f64 / 5.0;
            let z = z as f64 / 5.0;
           Polygon::new(vec![
                   (x, pdf(x,z), z),
                   (x + 0.2, pdf(x + 0.2,z), z),
                   (x + 0.2, pdf(x + 0.2,z + 0.2), z + 0.2),
                   (x, pdf(x,z + 0.2), z + 0.2),
               ],
               &HSLColor(240.0 / 360.0 - 240.0 / 360.0 * pdf(x,z) / 5.0,1.0,0.7))
        });

        chart.draw_series(series);
    }
    
    Ok(())
})

2D Guassian PDF
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-2.0
 
 
 
-1.0
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
 
-2.0
 
 
 
-1.0
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-2.0
 
 
 
-1.0
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
 
-2.0
 
 
 
-1.0
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0